# Retail Shelf: Price and Promotion Discrepancy Finder (BigQuery + Gemini)

This notebook demonstrates how to detect pricing/promotion mismatches by **combining image embeddings** (from shelf photos) with **structured pricing data** (CSV → BigQuery).  
This notebook is developed & tested to run in **BigQuery Studio → Notebooks** (or locally with copy‑paste).:

### What we’re solving
Retailers run thousands of **price & promo** campaigns weekly. In-store execution often drifts from plan:
- Wrong sign vs POS price (e.g., *2 för 30* but POS lists **29.95**).
- Wrong item/brand under sign.
- Outdated promo sign after expiry.
- Mixed promos within one display.
  
We **combine**:
1) **Image embeddings** of shelf photos (Gemini multimodal)  
2) **Text embeddings** of structured product pricing/promotions  
3) A **single natural-language query** (e.g., “gurka 2 för 30”) to retrieve relevant **images + products**, parse the sign (“2 for 30” → **15 SEK** each), and **flag discrepancies** by comparing against structured prices.

💡 Tip: The simplest way to run this notebook is by uploading it directly into BigQuery Studio → Notebooks. That’s where this workflow was developed and tested, so it should run seamlessly there.

## Prerequisites

- Google Cloud project with BigQuery and Vertex AI APIs enabled
- gcloud and bq CLIs installed and authenticated (gcloud auth login)
- A GCS bucket for sample images
- A CSV named pricing_promotions.csv (downloaded below)


## 0) Environment variables (edit as needed)

In [ ]:
import os
PROJECT_ID = os.environ.get('PROJECT_ID', 'sm-gemini-playground')
BQ_DATASET = os.environ.get('BQ_DATASET', 'retail_shelf')
BQ_LOCATION = os.environ.get('BQ_LOCATION', 'US')
CONN_ID = os.environ.get('CONN_ID', f'{BQ_LOCATION}.retail_shelf_conn_us')
GCS_BUCKET = os.environ.get('GCS_BUCKET', 'sm-gemini-pg-retail-semantic')
GCS_PREFIX_BASE = os.environ.get('GCS_PREFIX_BASE', 'retail_shelf')
GCS_PREFIX_GLOB = f'{GCS_PREFIX_BASE}/*'

OBJ_TABLE = os.environ.get('OBJ_TABLE', 'shelves')
IMG_EMB_TABLE = os.environ.get('IMG_EMB_TABLE', 'shelf_embeddings')
IMG_EMB_INDEX = os.environ.get('IMG_EMB_INDEX', 'idx_shelf_image_embeddings')
TXT_MODEL = os.environ.get('TXT_MODEL', 'text_embedding_model')
MM_MODEL = os.environ.get('MM_MODEL', 'multimodal_embedding_model')
PRICING_TABLE = os.environ.get('PRICING_TABLE', 'pricing_promotions')
PRICING_EMB_TABLE = os.environ.get('PRICING_EMB_TABLE', 'pricing_promotions_emb')
PRICING_VEC_VIEW = os.environ.get('PRICING_VEC_VIEW', 'pricing_promotions_vec')

print(PROJECT_ID, BQ_DATASET, BQ_LOCATION, CONN_ID)


In [ ]:
%env PROJECT_ID=$PROJECT_ID
%env BQ_DATASET=$BQ_DATASET
%env BQ_LOCATION=$BQ_LOCATION
%env CONN_ID=$CONN_ID
%env GCS_BUCKET=$GCS_BUCKET
%env GCS_PREFIX_BASE=$GCS_PREFIX_BASE
%env GCS_PREFIX_GLOB=$GCS_PREFIX_GLOB
%env OBJ_TABLE=$OBJ_TABLE
%env IMG_EMB_TABLE=$IMG_EMB_TABLE
%env IMG_EMB_INDEX=$IMG_EMB_INDEX
%env TXT_MODEL=$TXT_MODEL
%env MM_MODEL=$MM_MODEL
%env PRICING_TABLE=$PRICING_TABLE
%env PRICING_EMB_TABLE=$PRICING_EMB_TABLE
%env PRICING_VEC_VIEW=$PRICING_VEC_VIEW


## 1) Create dataset and Cloud Resource connection

In [ ]:
%%bash
set -euo pipefail

bq --location=${BQ_LOCATION} --project_id=${PROJECT_ID} mk -d ${BQ_DATASET} || echo "Dataset may already exist"

bq --project_id=${PROJECT_ID} mk --connection --location=${BQ_LOCATION} --connection_type=CLOUD_RESOURCE ${CONN_ID} || true

bq show --connection --location=${BQ_LOCATION} ${PROJECT_ID}.${CONN_ID} || true

# Grant required permissions to the BigQuery connection service account
# Replace SERVICE_ACCOUNT with the "serviceAccount:..." shown in the bq show output above

SERVICE_ACCOUNT="serviceAccount:YOUR-CONNECTION-SA from above output"


# Allow BigQuery connection SA to read from GCS
gcloud storage buckets add-iam-policy-binding gs://$GCS_BUCKET \
  --member="$SERVICE_ACCOUNT" \
  --role="roles/storage.objectViewer" \
  --project=$PROJECT_ID || true

# Allow BigQuery connection SA to call Vertex AI remote models
gcloud projects add-iam-policy-binding $PROJECT_ID \
  --member="$SERVICE_ACCOUNT" \
  --role="roles/aiplatform.user" || true


## 2) Copy demo images and CSV from GitHub

- Images: https://github.com/sijohn/bq_llm/tree/main/bq_multimodal_search/images

The images will be loaded to Google Cloud Storage Bucket and then to Bigquery Object Tables.

- CSV: https://github.com/sijohn/bq_llm/blob/main/bq_multimodal_search/pricing_promotions.csv

The CSV file will be used to create pricing data table in Bigquery


In [ ]:
%%bash
set -euo pipefail
if [ -d bq_llm ]; then (cd bq_llm && git pull --ff-only); else git clone https://github.com/sijohn/bq_llm.git; fi
gsutil -m cp -r bq_llm/bq_multimodal_search/images/* gs://${GCS_BUCKET}/${GCS_PREFIX_BASE}/

curl -L -o pricing_promotions.csv https://raw.githubusercontent.com/sijohn/bq_llm/main/bq_multimodal_search/pricing_promotions.csv

bq --location=${BQ_LOCATION} --project_id=${PROJECT_ID} load \
  --replace --autodetect --source_format=CSV --skip_leading_rows=1 \
  ${BQ_DATASET}.${PRICING_TABLE} ./pricing_promotions.csv


In [6]:
from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID, location=BQ_LOCATION)

## 3) Create Object Table over images

In [ ]:
sql = f'''
CREATE OR REPLACE EXTERNAL TABLE `{PROJECT_ID}.{BQ_DATASET}.{OBJ_TABLE}`
WITH CONNECTION `{PROJECT_ID}.{CONN_ID}`
OPTIONS ( object_metadata = 'SIMPLE', uris = ['gs://{GCS_BUCKET}/{GCS_PREFIX_GLOB}'] );
'''
job = client.query(sql)
job.result()
print("Object Table created:", f"{PROJECT_ID}.{BQ_DATASET}.shelves_v2")

## 4) Create multimodal embedding model and generate image embeddings

In [ ]:
sql = f'''
CREATE OR REPLACE MODEL `{PROJECT_ID}.{BQ_DATASET}.{MM_MODEL}`
REMOTE WITH CONNECTION `{PROJECT_ID}.{CONN_ID}`
OPTIONS ( endpoint = 'multimodalembedding@001' );
'''
job = client.query(sql)
job.result()
print("Remote model created:", f"{PROJECT_ID}.{BQ_DATASET}.{MM_MODEL}")

In [ ]:
sql = f'''
CREATE OR REPLACE TABLE `{PROJECT_ID}.{BQ_DATASET}.{IMG_EMB_TABLE}` AS
SELECT * FROM ML.GENERATE_EMBEDDING(
  MODEL `{PROJECT_ID}.{BQ_DATASET}.{MM_MODEL}`,
  TABLE `{PROJECT_ID}.{BQ_DATASET}.{OBJ_TABLE}`,
  STRUCT(TRUE AS flatten_json_output)
);
'''
job = client.query(sql)
job.result()
print("Embedding Table created:", f"{PROJECT_ID}.{BQ_DATASET}.{IMG_EMB_TABLE}")



## 5) Create a vector index for image embeddings (Optional)
Not needed if the sample image set is less than 5000

In [ ]:
sql = f'''
CREATE OR REPLACE VECTOR INDEX `{PROJECT_ID}.{BQ_DATASET}.{IMG_EMB_INDEX}`
ON `{PROJECT_ID}.{BQ_DATASET}.{IMG_EMB_TABLE}` (ml_generate_embedding_result)
OPTIONS ( index_type = 'IVF', distance_type = 'COSINE' );
'''
job = client.query(sql)
job.result()
print("Index created:", f"{PROJECT_ID}.{BQ_DATASET}.{IMG_EMB_INDEX}")

## 6) Text embeddings for pricing table and a clean view

In [ ]:
sql = f'''
CREATE OR REPLACE MODEL `{PROJECT_ID}.{BQ_DATASET}.{TXT_MODEL}`
REMOTE WITH CONNECTION `{PROJECT_ID}.{CONN_ID}`
OPTIONS ( ENDPOINT = 'gemini-embedding-001' );
'''
job = client.query(sql)
job.result()
print("Text Model created:", f"{PROJECT_ID}.{BQ_DATASET}.{TXT_MODEL}")



sql = f'''
CREATE OR REPLACE TABLE `{PROJECT_ID}.{BQ_DATASET}.{PRICING_EMB_TABLE}` AS
SELECT * EXCEPT(content)
FROM ML.GENERATE_EMBEDDING(
  MODEL `{PROJECT_ID}.{BQ_DATASET}.{TXT_MODEL}`,
  (
    SELECT TRIM(CONCAT(
      IFNULL(product_name,''), ' ', IFNULL(category,''), ' ', IFNULL(promo_text,''), ' ',
      IFNULL(FORMAT('listed %.2f', listed_price), ''), ' ',
      IFNULL(FORMAT('promo %.2f', promo_price), '')
    )) AS content,
    p.*
    FROM `{PROJECT_ID}.{BQ_DATASET}.{PRICING_TABLE}` p
  ),
  STRUCT(TRUE AS flatten_json_output, 'RETRIEVAL_DOCUMENT' AS task_type)
);
'''
job = client.query(sql)
job.result()
print("Pricing Embedding Table created:", f"{PROJECT_ID}.{BQ_DATASET}.{PRICING_EMB_TABLE}")

In [ ]:

sql = f'''
CREATE OR REPLACE VIEW `{PROJECT_ID}.{BQ_DATASET}.{PRICING_VEC_VIEW}` AS
SELECT * FROM `{PROJECT_ID}.{BQ_DATASET}.{PRICING_EMB_TABLE}`
WHERE ARRAY_LENGTH(ml_generate_embedding_result) = 3072;
'''
job = client.query(sql)
job.result()
print("Pricing View created:", f"{PROJECT_ID}.{BQ_DATASET}.{PRICING_VEC_VIEW}")


## 7) Enable table display (Colab only)

In [ ]:
# @title Enable data table display (Colab only)
import pandas as pd
try:
    %load_ext google.colab.data_table
except Exception as e:
    print('google.colab.data_table not available in this environment:', e)


In [27]:
# @title Util function to display images
import io
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf

def printImages(results):
    image_results_list = list(results)
    amt = len(image_results_list)
    if amt == 0:
        print('No rows to display.')
        return
    fig, axes = plt.subplots(nrows=amt, ncols=2, figsize=(20, max(4, amt*3)))
    if amt == 1:
        axes = [axes]
    fig.tight_layout()
    fig.subplots_adjust(hspace=0.5)
    for i in range(amt):
        gcs_uri = image_results_list[i][0]
        text = str(image_results_list[i][1])
        with tf.io.gfile.GFile(gcs_uri, 'rb') as f:
            img = Image.open(io.BytesIO(f.read())).convert('RGB')
        axes[i][0].axis('off')
        axes[i][0].imshow(img)
#         axes[i][0].set_title(gcs_uri, fontsize=8)
        axes[i][1].axis('off')
        axes[i][1].text(0, 0, text, fontsize=10, wrap=True)
    plt.show()


## 7) Example image searches (natural language)

In [ ]:
sql = f'''
DECLARE q STRING DEFAULT 'lindt chocolate bars';
WITH query_vec AS (
  SELECT ml_generate_embedding_result AS qvec
  FROM ML.GENERATE_EMBEDDING(
    MODEL `{PROJECT_ID}.{BQ_DATASET}.{MM_MODEL}`,
    (SELECT q AS content),
    STRUCT(TRUE AS flatten_json_output)
  )
)
SELECT base.uri,base.content_type
FROM VECTOR_SEARCH(
  TABLE `{PROJECT_ID}.{BQ_DATASET}.{IMG_EMB_TABLE}`, 'ml_generate_embedding_result',
  TABLE query_vec, query_column_to_search => 'qvec', top_k => 5, distance_type => 'COSINE'
);
'''
job = client.query(sql)
res = job.result()
printImages(res)


In [ ]:
sql = f'''
DECLARE q STRING DEFAULT 'cucumbers (gurka) 2 for 30';
WITH query_vec AS (
  SELECT ml_generate_embedding_result AS qvec
  FROM ML.GENERATE_EMBEDDING(
    MODEL `{PROJECT_ID}.{BQ_DATASET}.{MM_MODEL}`,
    (SELECT q AS content),
    STRUCT(TRUE AS flatten_json_output)
  )
)
SELECT base.uri,base.content_type
FROM VECTOR_SEARCH(
  TABLE `{PROJECT_ID}.{BQ_DATASET}.{IMG_EMB_TABLE}`, 'ml_generate_embedding_result',
  TABLE query_vec, query_column_to_search => 'qvec', top_k => 5, distance_type => 'COSINE'
);
'''
job = client.query(sql)
res = job.result()
printImages(res)


## 8) Demo : Finding Price discrepencie

PS: Solution is not perfect. In the next iteration, the image object table, will have an additional text embedding to describe the product_name. Otherwise the join between the price table and the image object table will be challenging.

In [ ]:
sql = f"""
-- Natural query typed ONCE by the auditor (passed as @q)
DECLARE q STRING DEFAULT 'Lindt chocolate with price discrepancy';

-- Parse "N for X" and compute per-unit sign price
DECLARE n INT64        DEFAULT CAST(IFNULL(REGEXP_EXTRACT(q, r'(\\d+)\\s*for'), '1') AS INT64);
DECLARE bundle FLOAT64 DEFAULT CAST(REGEXP_EXTRACT(q, r'for\\s*(\\d{{2,3}})') AS FLOAT64);
DECLARE sign_each_price FLOAT64 DEFAULT SAFE_DIVIDE(bundle, n);

WITH
img_q AS (
  SELECT ml_generate_embedding_result AS qvec
  FROM ML.GENERATE_EMBEDDING(
    MODEL `{PROJECT_ID}.{BQ_DATASET}.{MM_MODEL}`,
    (SELECT q AS content),
    STRUCT(TRUE AS flatten_json_output)
  )
),
txt_q AS (
  SELECT ml_generate_embedding_result AS qvec
  FROM ML.GENERATE_EMBEDDING(
    MODEL `{PROJECT_ID}.{BQ_DATASET}.{TXT_MODEL}`,
    (SELECT q AS content),
    STRUCT(TRUE AS flatten_json_output, 'RETRIEVAL_QUERY' AS task_type)
  )
),

-- 1) Top image hits
img_hits AS (
  SELECT base.uri, distance AS img_distance
  FROM VECTOR_SEARCH(
    TABLE `{PROJECT_ID}.{BQ_DATASET}.{IMG_EMB_TABLE}`, 'ml_generate_embedding_result',
    TABLE img_q, query_column_to_search => 'qvec',
    top_k => 10, distance_type => 'COSINE'
  )
),

-- 2) Product hits from structured table using TEXT embeddings
prod_hits AS (
  SELECT
    base.* EXCEPT (ml_generate_embedding_result),
    distance AS txt_distance
  FROM VECTOR_SEARCH(
    TABLE `{PROJECT_ID}.{BQ_DATASET}.{PRICING_EMB_TABLE}`, 'ml_generate_embedding_result',
    TABLE txt_q, query_column_to_search => 'qvec',
    top_k => 50, distance_type => 'COSINE'
  )
)

-- 3) Present: top images + best-matching products with discrepancy signals
SELECT
  ARRAY_TO_STRING(
    ARRAY(SELECT uri FROM img_hits ORDER BY img_distance LIMIT 3),
    '\\n'
  ) AS top_image_uris,
  q AS auditor_query,
  sign_each_price,
  product_name,
  category,
  listed_price,
  promo_text,
  promo_price,
  discrepancy_flag,
  ROUND(ABS(listed_price - sign_each_price), 2) AS delta_vs_sign_listed,
  ROUND(ABS(promo_price  - sign_each_price), 2) AS delta_vs_sign_promo,
  txt_distance
FROM prod_hits
ORDER BY delta_vs_sign_listed DESC, txt_distance
LIMIT 5;
"""

job = client.query(sql)
res = job.result()
res.to_dataframe()


## 13) Wrap up

- No manual image-to-product tagging
- Natural language powers both image and product retrieval

